In [ ]:
# seqeval のインストール
!pip install seqeval

In [ ]:
# seqeval を用いた精度計算
from seqeval.metrics import classification_report

y_true = [["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
y_pred = [["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"], # 各固有表現のトークン数が [0, 2, 1] -> [2, 1, 1]
          ["B-PER", "I-PER", "O"]]
print(classification_report(y_true, y_pred)) # 2つ目のみ正解となるので50%

In [ ]:
# モデルの出力を変換
import numpy as np

## tag への変換メソッド取得のため
from collections import defaultdict
from datasets import load_dataset, DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs,fracs):
  ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
  for split in ds:
    panx_ch[lang][split] = (
        ds[split]
        .shuffle(seed=0)
        .select(range(int(frac*ds[split].num_rows)))
    )

tags = panx_ch["de"]["train"].features["ner_tags"].feature
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}

## 変換関数
def align_predictions(predictions, label_ids):
  preds = np.argmax(predictions, axis=2)
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [], [] # 出力を入れるリスト

  for batch_idx in range(batch_size):
    example_labels, example_preds = [], []
    for seq_idx in range(seq_len):
      if label_ids[batch_idx, seq_idx] != -100: # -100はなかったことにする
        example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
        example_preds.append(index2tag[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)

  return preds_list, labels_list